# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all


You are using wmfdata v1.3.3, but v2.0.0 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/diversity_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2022, 11, 30)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [5]:
# Code to Suppress PySpark Warning messages
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
queries = {
    
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    },
    "global_north_previews": {
        "file": "queries/global_north_previews.hql",
        "engine": "hive"
    },
     "global_north_pageviews": {
        "file": "queries/global_north_pageviews.hql",
        "engine": "hive"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_south_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_north_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_north_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Combining and saving metrics

In [7]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)
new_metrics['gn_interactions'] = new_metrics.apply(lambda x: x['gn_previews'] + x['gn_pageviews'], axis=1)


In [8]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,gn_interactions,gn_pageviews,gn_previews,gs_interactions,gs_pageviews,gs_previews
month,,,,,,
2022-02-01,1.373794e+10,1.224834e+10,1.489596e+09,4.110316e+09,3.767028e+09,343288151.0
2022-03-01,1.451693e+10,1.290724e+10,1.609688e+09,4.407957e+09,4.051236e+09,356720322.0
2022-04-01,1.388053e+10,1.243007e+10,1.450457e+09,4.023481e+09,3.700706e+09,322775480.0
2022-05-01,1.405023e+10,1.259418e+10,1.456056e+09,4.102891e+09,3.771585e+09,331305888.0
2022-06-01,1.315583e+10,1.184037e+10,1.315461e+09,3.923353e+09,3.613206e+09,310147238.0
2022-07-01,1.370182e+10,1.243396e+10,1.267863e+09,4.128744e+09,3.821433e+09,307311517.0
2022-08-01,1.417988e+10,1.286435e+10,1.315527e+09,4.686663e+09,4.367812e+09,318850733.0
2022-09-01,1.456143e+10,1.318547e+10,1.375966e+09,5.278138e+09,4.948280e+09,329857128.0
2022-10-01,1.509622e+10,1.368440e+10,1.411820e+09,5.367894e+09,5.040110e+09,327784326.0


In [9]:
metrics.to_csv(FILENAME, sep="\t")